## importing Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
project_DIR=r"F:\DATA_SCIENCE\Machile_learing_projects\Campusx_projects\Flight_Price_Prediction"
DATA_DIR='Data'

In [14]:
def get_data(name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_DIR,DATA_DIR,file_name)
    return pd.read_csv(file_path)
    

In [16]:
# Reading the data
flights=get_data("flight_price")
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [17]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The Dataset contains 10,683 rows and in Route and Total_stops column there are some missing value
- We need to change the datatpe for some feature is in appropiate format.

# Preliminary analysis

# check for data types

In [21]:
flights.head(2)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662


In [20]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

# Checking for dulicated

In [24]:
flights.duplicated().sum()

220

In [28]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(['Airline',"Date_of_Journey","Source","Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


# Observations
- The type of date_of_journy is in object dtype and we need to change it
- The type of 'Dep_Time' should be change into datetime and arival_time should be changed into datetime
- The type of 'Duration ' is mixed and we need to change it in appropiate value
- The 'Total_Stops' should be in numeric type
- There are 220 rows are dublicated in our dataset so we need to remove these

# Detailed Analysis

In [57]:
## Airline

In [59]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [62]:
(
    flights.Airline
    .str.replace('Premium economy',"")
    .str.replace('Business','')
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

In [79]:
# Date_of_journey
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [81]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [89]:
## Source
flights.Source.unique() # source is fine

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [91]:
#destination
flights.Destination.unique() # it is also fine

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [ ]:
#Route
# the same information is present in otehr columns also so we need to remove the route column

In [97]:
#Dep_time
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [100]:
(
    flights
    .Dep_Time
    #filtering those rows that might contains characters excepet number and : 
    .loc[lambda ser: ser.str.contains('[^0-9:]')]
) # it means all the vlaue are in the form of number:number only if empy list come

Series([], Name: Dep_Time, dtype: object)

In [104]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\rajkr\AppData\Local\Temp\ipykernel_19984\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [ ]:
#arrival time

In [107]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [117]:
(
    flights
    .Arrival_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(-1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

In [120]:
#Duration

In [121]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [124]:
# checking whihc rows have only hours and minutes
(
    flights.Duration
    .loc[lambda ser:~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [126]:
(
    flights.Duration
    .loc[lambda ser:~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [127]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [ ]:
# extracing the number from hour and multiply with 60 so that it can be consider as minutes and then we need to extract the numer from minutes and add with hour and make a seprate column

In [152]:
(
    flights.Duration
    .drop(index=[6474])
    .str.split(" ",expand=True)
    .set_axis(['hours','minutes'],axis=1)
    .assign(
        hours=lambda df_:(
            df_
            .hours
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_:
               df_
               .minutes
               .str.replace('m',"")
                .fillna("0")
                .astype(int)
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [155]:
(
    flights.Duration
    .drop(index=[6474])
    .str.split(" ",expand=True)
    .set_axis(['hours','minutes'],axis=1)
    .assign(
        hours=lambda df_:(
            df_
            .hours
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_:
               df_
               .minutes
               .str.replace('m',"")
                .fillna("0")
                .astype(int)
    )
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()
    #validating the data with original 
    .join(flights.Duration.drop(index=[6474]))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [168]:
#total_stops
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [174]:
(
    flights.Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)
    .pipe(lambda ser:pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [181]:
#additional info

In [183]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [ ]:
# the obeservation the index 6474 has duration of 5 minutes which is wrong and we need to remove it

# cleaning operations with functional programming

In [54]:
#selecting the features which have data type object
flights.select_dtypes(include='O').columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

In [164]:
def convert_to_minutes(series):
        return (
        series
        .str.split(" ",expand=True)
        .set_axis(['hours','minutes'],axis=1)
        .assign(
            hours=lambda df_:(
                df_
                .hours
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minutes=lambda df_:
            df_
            .minutes
            .str.replace('m',"")
            .fillna("0")
            .astype(int)
        )
        .sum(axis=1)
)

In [184]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .assign(**{
            col:df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
        })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_:(
                df_
                .airline
                .str.replace('Premium economy',"")
                .str.replace('Business','')
                .str.title()
        ),
            date_of_journey=lambda df_:pd.to_datetime(df_.date_of_journey,dayfirst=True),
            dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_:df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_:
                       df_
                       .total_stops
                       .replace("non-stop","0")
                        .str.replace(" stops?","",regex=True)
                        .pipe(lambda ser:pd.to_numeric(ser))
        ),
        additional_info=lambda df_:df_.additional_info.replace("No info","No Info")
        .drop(columns='route')
    )

SyntaxError: invalid syntax (1396150500.py, line 29)

In [186]:
flights_cleaned=clean_data(flights)#.dtypes#.airline.unique()

C:\Users\rajkr\AppData\Local\Temp\ipykernel_19984\4280144057.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\rajkr\AppData\Local\Temp\ipykernel_19984\4280144057.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


In [187]:
flights_cleaned.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No info,13302


In [ ]:
# pipe is a method in pandas that can se apply on any dataframes and series

# Split the data

In [189]:
flights_final = flights_cleaned.sample(1000)

In [190]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [191]:

X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


# Export the subsets

In [194]:
def export_data(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(project_DIR, DATA_DIR, file_name)

	X.join(y).to_csv(file_path, index=False)

	return pd.read_csv(file_path).head()

In [195]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-12,Delhi,Cochin,02:00:00,07:45:00,345,1.0,No info,6386
1,Air Asia,2019-05-21,Kolkata,Banglore,22:40:00,10:40:00,720,1.0,No info,9719
2,Indigo,2019-04-01,Kolkata,Banglore,17:15:00,19:50:00,155,0.0,No info,4174
3,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,07:40:00,1420,1.0,No info,14815
4,Air India,2019-03-15,Chennai,Kolkata,11:40:00,13:55:00,135,0.0,No info,7082


In [196]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-05-24,Kolkata,Banglore,16:50:00,07:55:00,905,1.0,No info,8576
1,Jet Airways,2019-05-27,Delhi,Cochin,07:05:00,12:35:00,330,1.0,In-flight meal not included,12898
2,Indigo,2019-06-18,Chennai,Kolkata,07:55:00,10:15:00,140,0.0,No info,3850
3,Jet Airways,2019-06-03,Delhi,Cochin,20:55:00,12:35:00,940,1.0,No info,14714
4,Jet Airways,2019-05-09,Kolkata,Banglore,09:35:00,23:35:00,840,1.0,No info,12121


In [197]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-03-12,Banglore,New Delhi,11:05:00,22:10:00,665,2.0,No info,11948
1,Indigo,2019-05-09,Kolkata,Banglore,08:10:00,13:00:00,290,1.0,No info,5069
2,Vistara,2019-04-01,Kolkata,Banglore,17:00:00,23:25:00,385,1.0,No info,8610
3,Indigo,2019-05-21,Banglore,Delhi,18:25:00,21:20:00,175,0.0,No info,4823
4,Indigo,2019-04-24,Kolkata,Banglore,09:35:00,12:20:00,165,0.0,No info,4804
